In [1]:
from whisk.kitchenai_sdk.schema import ChatInput, ChatResponse
from whisk.kitchenai_sdk.kitchenai import KitchenAIApp


kitchen = KitchenAIApp(namespace="streaming-demo")



In [2]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.tools import FunctionTool
import asyncio

# Define a simple tool for demonstration
def calculator(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

# Create a tool instance
calc_tool = FunctionTool.from_defaults(fn=calculator)


# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo")

# Non-streaming Agent
# Create the agent with tools
agent_b = ReActAgent.from_tools(
    tools=[calc_tool],
    llm=llm,
    verbose=True
)
    


# Streaming Agent
    # Create the agent with streaming enabled
agent = ReActAgent.from_tools(
    tools=[calc_tool],
    llm=llm,
    verbose=True
)
    



In [3]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

async for token in response.async_response_gen():
    print(token, end="")

> Running step 93610a8f-e24b-486f-b0ca-2264a2833a52. Step input: What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: calculator
Action Input: {'a': 121, 'b': 8}
Observation: 129
> Running step f7ea3041-7c02-454e-90e5-af3f6b458724. Step input: None
 129

Once upon a time, in a cozy little burrow nestled beneath a large oak tree, there lived a group of adventurous mice. The leader of the group, a wise old mouse named Whiskers, had just discovered a hidden treasure of 129 shiny acorns. Excited by their newfound wealth, the mice decided to throw a grand feast to celebrate their good fortune. As the moon rose high in the sky, the mice danced and feasted on delicious acorn treats, their tiny paws tapping to the rhythm of the night. And so, the mice lived happily ever after, their bellies full and their hearts content in their cozy 

In [4]:
from typing import AsyncGenerator


In [11]:
@kitchen.chat.handler("streaming-response")
async def streaming_response(input: ChatInput) -> AsyncGenerator[ChatResponse, None]:
    response = await llm.astream_complete(
        input.messages[-1].content
    )


    async for delta in response:
        yield ChatResponse(content=delta.delta, role="assistant")


In [ ]:
# Launch Whisk server
from whisk.config import WhiskConfig, ServerConfig
from whisk.router import WhiskRouter

config = WhiskConfig(server=ServerConfig(type="fastapi"))
router = WhiskRouter(kitchen_app=kitchen, config=config)

# Run the Whisk server in the notebook
router.run_in_notebook(host="0.0.0.0", port=8000)

Shutting down existing Whisk server...
Whisk server stopped.
Whisk server started on http://0.0.0.0:8000 (in background)


INFO:     Started server process [690865]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57072 - "OPTIONS /v1/chat/completions HTTP/1.1" 200 OK
INFO:     127.0.0.1:57076 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     127.0.0.1:57076 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     127.0.0.1:57076 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     127.0.0.1:38258 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
router.stop_in_notebook()

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [690865]


Shutting down existing Whisk server...
Whisk server stopped.
